In [1]:
pip install transformers torch colbert datasets pandas scikit-learn


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for colbert: filename=Colbert-0.40-py3-none-any.whl size=70698 sha256=7529b27995253b05108122c76778c579af2a0781ce8f41d9705e905a3d932dd2
  Stored in directory: c:\users\davog\appdata\local\pip\cache\wheels\1b\05\58\eb3ba149f8580abf723b35f24341b5dd6c64bbfbd489eee9a9
Successfully built colbert
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install git+https://github.com/stanford-futuredata/ColBERT.git


  Cloning https://github.com/stanford-futuredata/ColBERT.git to c:\users\davog\appdata\local\temp\pip-req-build-tmbg8tgw
  Resolved https://github.com/stanford-futuredata/ColBERT.git to commit 7067ef598b5011edaa1f4a731a2c269dbac864e4
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for colbert-ai: filename=colbert_ai-0.2.20-py3-none-any.whl size=117283 sha256=6cb88bd3ee8577be190cb2dba5c91dd0490bbc7a2615be3f5fa9f18681379ee4
  Stored in directory: C:\Users\davog\AppData\Local\Temp\pip-ephem-wheel-cache-9ry8hr6c\wheels\3d\ab\13\3e4a9504de71af66b661bfbe0105363f9c2b422f63aa264171
Successfully built colbert-ai
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/stanford-futuredata/ColBERT.git 'C:\Users\davog\AppData\Local\Temp\pip-req-build-tmbg8tgw'


In [5]:
import pandas as pd
article_df = pd.read_csv('./src/article_scraping/data/articles_with_text.csv')
print(f"Total articles in CSV: {len(article_df)}")


Total articles in CSV: 4354


In [2]:
import pandas as pd

# Load data
article_df = pd.read_csv('./src/article_scraping/data/articles_with_text.csv')

# Create a unique document ID for each article
article_df['doc_id'] = range(len(article_df))

# Replace newline characters in article text to avoid formatting issues
article_df['article_text_raw'] = article_df['article_text_raw'].str.replace(r'\n', ' ', regex=True)

# Separate rows with text from those without
with_text = article_df[~article_df['article_text_raw'].isnull() & (article_df['article_text_raw'].str.strip() != '')]
without_text = article_df[article_df['article_text_raw'].isnull() | (article_df['article_text_raw'].str.strip() == '')]

# Prepare the formatted text column for rows with text
with_text['formatted_text'] = with_text['doc_id'].astype(str) + '\t' + with_text['article_text_raw']

# Save the .tsv file (only articles with text)
with_text['formatted_text'].to_csv('./src/article_scraping/data/collection.tsv', index=False, header=False, sep='\t')

# Print a summary of the process
print(f"Articles at start: {len(article_df)}")
print(f"Articles with text: {len(with_text)}")
print(f"Articles without text: {len(without_text)}")




C:\Users\davog\AppData\Local\Temp\ipykernel_20536\2489671770.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_text['formatted_text'] = with_text['doc_id'].astype(str) + '\t' + with_text['article_text_raw']


Articles at start: 4354
Articles with text: 4268
Articles without text: 86


In [ ]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Indexer
import logging
import os

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

if __name__ == '__main__':
    with Run().context(RunConfig(nranks=1, experiment="dsan5400_project")):
        
        # Configure ColBERT
        config = ColBERTConfig(
            nbits=2,
            root="./experiments",
            bsize=32 
        )
        
        # Initialize Indexer
        indexer = Indexer(checkpoint="checkpoints/colbertv2.0", config=config)
        
        # Wrapper to log progress
        def logging_index(name, collection, **kwargs):
            if not os.path.exists(collection):
                logger.error(f"Collection file '{collection}' does not exist.")
                return
            
            # Count total documents for progress tracking
            with open(collection, 'r', encoding='utf-8') as file:
                total_docs = sum(1 for _ in file)
            logger.info(f"Starting indexing for {total_docs} documents.")
            
            processed_docs = 0
            batch_size = config.bsize  
            
            with open(collection, 'r', encoding='utf-8') as file:
                batch = []
                for line in file:
                    batch.append(line.strip())
                    processed_docs += 1
                    
                    # Process batch once it reaches the batch size
                    if len(batch) == batch_size:
                        logger.info(f"Processing batch {processed_docs}/{total_docs}...")
                        indexer.index(name=name, collection=batch, **kwargs)
                        batch = []  # Clear batch after processing
                    
                # Process remaining lines
                if batch:
                    logger.info(f"Processing final batch {processed_docs}/{total_docs}...")
                    indexer.index(name=name, collection=batch, **kwargs)
            
            logger.info("Indexing complete!")

        # Run the logging wrapper
        logging_index(
            name="dsan5400_project.nbits=2",
            collection="./src/article_scraping/data/collection.tsv",
            overwrite=True
        )


2024-12-06 11:12:00,092 - Starting indexing for 90984 documents.
2024-12-06 11:12:00,096 - Processing batch 32/90984...




[Dec 06, 11:12:00] #> Note: Output directory c:\Users\davog\Documents\GitHub\DSAN5400\DSAN5400_final_project\experiments\dsan5400_project\indexes/dsan5400_project.nbits=2 already exists


#> Starting...
